# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request as urllib2
import requests
from bs4 import BeautifulSoup
import pymorphy2

## Скрапинг сайта Gismeteo.
Воспользуемся бибилиотекой **BeautifulSoup**, чтобы достать с html страницы сайта gismeteo необходимую информацию о погоде в городе Москва на ближайшие 10 дней. 


**BeautifulSoup** - это парсер для синтаксического разбора файлов HTML/XML. Он поддерживает простые и естественные способы навигации, поиска и модификации дерева синтаксического разбора. 
Основным методoм для поиска необходимых данных был **findAll()** - метод обхода дерева, который начинает с заданной точки и ищет все объекты, соответствующие заданным критериям. С его помощью находился тег с нужным классом.

Так же по найденному тегу применялся метод **get_text**. С его помощью вытаскивался текст который хранился внутри тега.

In [2]:
# у бибилиотеки BeautifulSoup возникли проблемы с подключением к сайту гисметео. Решим их с помощью библиотеки requests. 
r = requests.get('https://www.gismeteo.ru/weather-moscow-4368/10-days/', 
                 headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.3"})

In [3]:
meteo = "https://www.gismeteo.ru/weather-moscow-4368/10-days/"
page = urllib2.urlopen(meteo)

soup = BeautifulSoup(page, 'lxml')

#soup.prettify()

In [4]:
def beautify_nums(s):
    return float(s.strip().replace('−', '-').replace(',','.'))

In [5]:
months = {'янв': '01',
          'фев': '02',
          'мар': '03',
          'апр': '04',
          'мая': '05',
          'июн': '06',
          'июл': '07',
          'авг': '08',
          'сен': '09',
          'окт': '10',
          'ноя': '11',
          'дек': '12'}
days = {'пн': 'понедельник',
        'вт': 'вторник',
        'ср': 'среда', 
        'чт': 'четверг',
        'пт': 'пятница',
        'сб': 'суббота',
        'вс': 'воскресенье'}

**Найдем нужную ветку в нашем дереве тегов, чтобы дальше вытащить информации из ее обьектов.**
* date - дата. Для корректного представления даты были также использованы словари months и days.
* max_temp - максимальная температура в рассматриваемый день.
* min_temp - минимальная температура в рассматриваемый день.
* wind - уровень ветра.
* rainfall уровень осадков.
* raw_comments - подписи к картинкам на сайте, которые помогут разнообразить описание погоды в задании 2.


In [6]:
# find right table with data
right_table = soup.find(class_='forecast_frame')

list_data = []
df_list = []
comments = dict()
num_days = 10
current_city= 'Москва'

# get data by classes
converted_date = ''
month = ''
date = right_table.findAll(class_='w_date')
max_temp = right_table.findAll(class_='maxt')
min_temp = right_table.findAll(class_='mint')
wind = right_table.findAll(class_='w_wind')
rainfall = right_table.findAll(class_='w_precipitation__value')
raw_comments = right_table.findAll(class_='tooltip')
month += '.' + months[date[0].get_text().split()[-1]]

In [7]:
for i in range(num_days):
    converted_date += days[date[i].get_text().split()[0].lower()] + ' ' +  date[i].get_text().split()[1] + month
    list_data.append(converted_date)
    list_data.append(beautify_nums(max_temp[i].get_text().strip().replace(',','.')))
    list_data.append(beautify_nums(min_temp[i].get_text().strip().replace(',','.')))
    list_data.append(beautify_nums(wind[i].get_text()))
    list_data.append(beautify_nums(rainfall[i].get_text()))
    comments[converted_date] = raw_comments[i]['data-text'].lower()
    
    df_list.append(list_data)
    list_data = []
    converted_date = ''

In [8]:
comments

{'воскресенье 18.02': 'пасмурно,  снег',
 'воскресенье 25.02': 'малооблачно',
 'вторник 20.02': 'пасмурно, небольшой снег',
 'понедельник 19.02': 'пасмурно, небольшой снег',
 'понедельник 26.02': 'ясно',
 'пятница 23.02': 'пасмурно, небольшой снег',
 'среда 21.02': 'переменная облачность, небольшой снег',
 'суббота 17.02': 'пасмурно, небольшой снег',
 'суббота 24.02': 'облачно',
 'четверг 22.02': 'переменная облачность, небольшой снег'}

In [9]:
tmp = np.rollaxis(np.asarray(df_list), 1, 0)
df = pd.DataFrame(tmp[1:], columns=tmp[0], index=['минимальная температура', 'максимальная температура', 'скорость ветра', 'уровень осадков'])
df

,суббота 17.02,воскресенье 18.02,понедельник 19.02,вторник 20.02,среда 21.02,четверг 22.02,пятница 23.02,суббота 24.02,воскресенье 25.02,понедельник 26.02
минимальная температура,-4.0,-4.0,-3.0,-6.0,-10.0,-11.0,-11.0,-12.0,-16.0,-17.0
максимальная температура,-5.0,-6.0,-6.0,-9.0,-14.0,-17.0,-16.0,-18.0,-20.0,-23.0
скорость ветра,3.0,7.0,6.0,8.0,8.0,7.0,5.0,8.0,9.0,9.0
уровень осадков,0.75,3.2,1.5,0.8,0.1,0.1,0.2,0.0,0.0,0.0


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

## Построение генeратора с применением библиотеки Pymorphy2.
Pymorhpy2 - морфологический анализатор, который являлся основыным инструментом при выполнении данного задания так как благодаря ему можно:
* ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
* согласовывать слова с числительными.

In [10]:
morph = pymorphy2.MorphAnalyzer()

In [11]:
word = 'яблоко'
parsed_word = morph.parse(word)[0]
print(1, parsed_word.make_agree_with_number(1).word) # согласование слова с числительным 1
print(2, parsed_word.make_agree_with_number(2).word) # согласование слова с числительным 2
print(3, parsed_word.make_agree_with_number(5).word) # согласование слова с числительным 5

1 яблоко
2 яблока
3 яблок


In [12]:
print(parsed_word.inflect({'gent'}).word) # слово в родительном падеже

яблока


Ниже приведен пример решения аналогичной задачи генерации текстов: генератор отчета о том, сколько и каких фруктов съел Вася.

In [13]:
def eating(n, fruit):
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join(['Вася съел', str(n), parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [14]:
eating(5, 'яблоко')

'Вася съел 5 яблок .'

In [15]:
eating(4, 'груша')

'Вася съел 4 груши .'

Пример более сложного генератора: фрукты могут есть не только мальчики, но и девочки, то есть, нужно не только согласовать числительное с существительным, но и поставить глагол в нужную форму.

In [16]:
def eating(name, n, fruit):
    name_tag = morph.parse(name)[0].tag
    if 'masc' in name_tag:
        gender = 'masc'
    if 'femn' in name_tag:
        gender = 'femn'
    verb = morph.parse("съесть")[0].inflect({'perf', gender,'sing','past','indc'}).word
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join([name, verb, str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [17]:
eating('Маша', 3, 'вишня')

'Маша съела 3 вишни .'

### Функция Weather Forecast.
** Составляет сообщения с прогнозом погоды на основе данных с сайта Gismeteo оформленных в датафрейм, построенный в задании 1. **
Аргументы:
* day_1 - в шаблонах выше это день1. По дефолту ставится следующий день после текущего.
* day_2 - в шаблонах выше это день2. По дефолту ставится текущий день. 


(day_1 хронологически после day_2)

Для составления корректных предложений используется класс **pymorphy2.MorphAnalyzer()**, который произвдит морфологический анализ слова и выделяет его основные характеристики, которые впоследствии мы можем изменять для того чтобы слово подходило под контекст и свою позицию в предложении. Также используется метод **parse**,который представляет морфологические разборы слова и атрибуты:
* inflect({'необходимый падеж'}) - ставит слово в необходимый падеж.
* make_agree_with_number(число с которым согласуется слово) - согласует слово с числительным.


In [60]:
def weather_forecast(day_1=None, day_2=None):
    #day_1 is after day_2
    if day_1 is None:
        day_1 = df.columns[1]
    if day_2 is None:
        day_2 = df.columns[0]
        
    date_1 = day_1.split()[1]
    date_2 = day_2.split()[1]
    
    transf_day_1 = morph.parse(day_1.split()[0])[0]
    transf_day_2 = morph.parse(day_2.split()[0])[0]
    transf_city = morph.parse(current_city)[0]
    
    if transf_day_1.word == 'вторник':
        prep_1 = 'Во'
    else:
        prep_1 = 'В'
        
    if transf_day_2.word in ['вторник', 'среда']:
        prep_2 = 'со'
    else:
        prep_2 = 'с'
    
    #temperature message
    #В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    
    temp_1 = (float(df[day_1]['максимальная температура']) + float(df[day_1]['минимальная температура']))/2
    
    temp_2 = (float(df[day_2]['максимальная температура']) + float(df[day_2]['минимальная температура']))/2
    
    delta = int(temp_1 - temp_2)
    
    if delta >= 0:
        changes = 'потеплеет на'
    else:
        changes = 'похолодает на'
    
    
    s_1 = ' '.join([prep_1, transf_day_1.inflect({'accs'}).word, date_1,
                  'в', transf_city.inflect({'loct'}).word.capitalize(),
                  changes, str(abs(delta)), morph.parse('градус')[0].make_agree_with_number(abs(delta)).word,
                  'по сравнению', prep_2, transf_day_2.inflect({'ablt'}).word, date_2]) + '.'
   
    #wind message
    #Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    
    wind_1 = float(df[day_1]['скорость ветра'])
              
    wind_2 = float(df[day_2]['скорость ветра'])
    
    delta = int(wind_1 - wind_2)

    s_2 = ' '.join(['Скорость ветра изменится на', str(abs(delta)), 
                    morph.parse('единицу')[0].make_agree_with_number(abs(delta)).word,
                    prep_1.lower(), transf_day_1.inflect({'ablt'}).word, date_1,
                   'по сравнению', prep_2, transf_day_2.inflect({'ablt'}).word, date_2]) + '.'
    
    #precipitation message
    #Уровень осадков повысится / понизится на X единиц за Y дней. 
    
    prec_level_1 = float(df[day_1]['уровень осадков'])
    
    prec_level_2 = float(df[day_2]['уровень осадков'])
    
    delta = float(prec_level_1 - prec_level_2)
    
    if delta > 1:
        changes = 'повысится на'
    elif delta < -1:
        changes = 'понизится на'
    else:
        changes = 'не изменится'

    num_days = abs(int(float(day_1.split()[1]) - float(day_2.split()[1])))
    
    if changes == 'не изменится':
        s_3 = ' '.join(['Уровень осадков', changes, 
                        'за', str(num_days), morph.parse('день')[0].make_agree_with_number(num_days).word]) + '.'
    else:
        s_3 = ' '.join(['Уровень осадков', changes, str(abs(delta)), 
                        morph.parse('единицу')[0].make_agree_with_number(abs(delta)).word, 
                        'за', str(num_days), morph.parse('день')[0].make_agree_with_number(num_days).word]) + '.'
    #comments message
    #В день2 в НазваниеГорода будет (описание погоды) по сравнению с день1 когда в НазваниеГорода (описание погоды)
    
    if comments[day_1] != comments[day_2]:
        s_4 = ' '.join([prep_1, transf_day_1.inflect({'accs'}).word, date_1,
                        'в', transf_city.inflect({'loct'}).word.capitalize(), 
                        'будет', ' и'.join(comments[day_1].split(',')), 'по сравнению',
                        prep_2, transf_day_2.inflect({'ablt'}).word, date_2,
                       'когда в', transf_city.inflect({'loct'}).word.capitalize(),
                        ' и'.join(comments[day_2].split(','))]) + '.'
    else:
        s_4 = ' '.join([prep_1, transf_day_1.inflect({'accs'}).word,
                        'в', transf_city.inflect({'loct'}).word.capitalize(), date_2,
                        'будет', ' и'.join(comments[day_1].split(',')),
                        'так же как и', prep_1.lower(), transf_day_2.inflect({'accs'}).word, date_2]) + '.'

    #final forecast message
    s = '\n'.join([s_1, s_2, s_3, s_4])
    
    return s

In [61]:
for i in range(1, len(df.columns)//2 + 1):
    day_1 = df.columns[-i]
    day_2 = df.columns[i - 1]
    print('-----------------------------------FORECAST_%d-----------------------------------'%i)
    print(weather_forecast(day_1, day_2))

-----------------------------------FORECAST_1-----------------------------------
В понедельник 26.02 в Москве похолодает на 15 градусов по сравнению с субботой 17.02.
Скорость ветра изменится на 6 единиц в понедельником 26.02 по сравнению с субботой 17.02.
Уровень осадков не изменится за 9 дней.
В понедельник 26.02 в Москве будет ясно по сравнению с субботой 17.02 когда в Москве пасмурно и небольшой снег.
-----------------------------------FORECAST_2-----------------------------------
В воскресенье 25.02 в Москве похолодает на 13 градусов по сравнению с воскресеньем 18.02.
Скорость ветра изменится на 2 единицы в воскресеньем 25.02 по сравнению с воскресеньем 18.02.
Уровень осадков понизится на 3.2 единицы за 7 дней.
В воскресенье 25.02 в Москве будет малооблачно по сравнению с воскресеньем 18.02 когда в Москве пасмурно и  снег.
-----------------------------------FORECAST_3-----------------------------------
В субботу 24.02 в Москве похолодает на 10 градусов по сравнению с понедельником

Выведем еще раз табличку с погодными данными для сравнения текстового прогноза.

In [20]:
df

,суббота 17.02,воскресенье 18.02,понедельник 19.02,вторник 20.02,среда 21.02,четверг 22.02,пятница 23.02,суббота 24.02,воскресенье 25.02,понедельник 26.02
минимальная температура,-4.0,-4.0,-3.0,-6.0,-10.0,-11.0,-11.0,-12.0,-16.0,-17.0
максимальная температура,-5.0,-6.0,-6.0,-9.0,-14.0,-17.0,-16.0,-18.0,-20.0,-23.0
скорость ветра,3.0,7.0,6.0,8.0,8.0,7.0,5.0,8.0,9.0,9.0
уровень осадков,0.75,3.2,1.5,0.8,0.1,0.1,0.2,0.0,0.0,0.0


### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

1) **В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?**

* Описание курса валют:

  Шаблон: различные валюты (доллар,евро, рубль и т д), соотношения между ними (между долларом и евро, например),     числовые значения
  Пример: *Курс доллара понизился до 56 рублей*



* Навигация:

  Шаблон: расстояние от пункта А до пунтка Б, время в пути
  Пример: *Путь от Москвы до Санкт-Петербурга составляет 709 км, приблизительное время поездки на машине 9ч 7м.*



2) **Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? **

* Добавить примечания

  Пример: Если в каком-то городе ожидается дождь, добавить сообщение «Не забудьте взять зонт». Или если значение   скорости ветра выше какого-то порога предупреждать о возможности шторма, предложить остаться дома.


  **Как знание синтаксической структуры предложения может помочь в этой задаче?**

Синтаксическая структура:
https://dic.academic.ru/dic.nsf/enc_mathematics/5085/..
https://habrahabr.ru/post/148124/

Исходя из этого определения знание синтаксической структуры может помочь при составлении сложных шаблонов, в которых присутствует большое количество согласований между словами, а также поможет построить предложение корректно с точки зрение рассматриваемого языка.

## Домашнее задание выполнили: Бобровских Глеб, Багиян Нерсес, Богданова Екатерина.

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 17.02.2018.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 1: Анна Лапидус (email: anyalapidus@list.ru, telegram: @alpids).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт:

ИАД-1 : VNYQlGd

ИАД-2 : wDEa8ge

ИАД-3 : SviK5dd

ИАД-4 : dzdD8YG


 У вас появится курс “ВШЭ > Прикладные задачи анализа данных” в разделе “Посещает курсы”.
* Перейдите по ссылке “ВШЭ > Прикладные задачи анализа данных (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/281